In [1]:
import glmnet_python
from glmnet import glmnet

# Import relevant modules and setup for calling glmnet
%reset -f
%matplotlib inline

import time
import sys
import os
import re
from itertools import compress
from glob import glob
import pickle

import scipy, importlib, pprint, matplotlib.pyplot as plt, warnings
from scipy.io import loadmat

import numpy as np
from glmnet import glmnet; from glmnetPlot import glmnetPlot
from glmnetPrint import glmnetPrint; from glmnetCoef import glmnetCoef; from glmnetPredict import glmnetPredict
from cvglmnet import cvglmnet; from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot; from cvglmnetPredict import cvglmnetPredict

# glmnet has many deprecation warnings from using scipy.* instad of numpy.*
warnings.filterwarnings("ignore", category=DeprecationWarning) 


In [2]:
# Define constants
ncores = 28 # 56
names = ['DA', '5HT', 'pH', 'NE']
data_prefix = '/mnt/nfs/proj/in-vitro/iterate/results_014/model_style_008/training-A-RBE-97Hz/'
results_prefix = '/mnt/nfs/proj/in-vitro/Leonardo/glmnet/fits/'
good_probes = ['MMA003W01R04', 'MMA004W01R04', 'MMA013W01R04', 'MMA018W01R04', 'MMA019W01R04', 
          'MMA022W01R04', 'MMA023W01R04', 'MMA024W01R04', 'MMA025W01R04']
filter_files = lambda x : x.find('500kohm') == -1 # only get files without the 500kohms resistor

nrecords_per_session = 300 # * 4 * 5
val_ratio = .1
# val_probe = 0 # use this probe for validation, ignores val_split
val_probe = -1 # split data randomly


In [3]:
# Load data

def load_matlab(vfiles, nrecords_per_session=150, proj_y=lambda x: x):
    yv = []
    yl = []
    for v in vfiles:
#         print()
#         print(v)
        bv = np.array(loadmat(v)['voltammograms'])
        bl = np.array(loadmat(v.replace('voltammograms.mat', 'labels.mat'))['labels'])

        for (xv, xl) in zip(bv,bl):
            a = xv[0][:,:nrecords_per_session].T
            a = np.apply_along_axis(np.diff, axis=1, arr=a) 
            b = xl[0][:nrecords_per_session,:4]
            b = np.apply_along_axis(proj_y, axis=1, arr=b) 
            yv.append(a.astype(np.float64))
            yl.append(b.astype(np.float64))

    x = np.vstack(yv)
    y = np.vstack(yl)
    
    return x, y

start_time = time.time()

voltammograms = glob(data_prefix + '/*MMA*/voltammograms.mat')
print('number of voltammograms files %d'%len(voltammograms))
voltammograms = list(compress(voltammograms, [filter_files(x) for x in voltammograms]))
print('after filter %d'%len(voltammograms))

if good_probes:
    keep = np.array([False]*len(voltammograms))
    for probe in good_probes:
        keep = keep | np.array([x.find(probe) > -1 for x in voltammograms])
    voltammograms = list(compress(voltammograms, keep))
    print('after removing bad probes %d'%len(voltammograms))

if val_probe < 0:
    print('number of train/val files %d'%len(voltammograms))
    x, y = load_matlab(voltammograms, nrecords_per_session)

    idxs = np.random.permutation(x.shape[0])
    lim = int(x.shape[0]*(1-val_ratio))
    d1idx = idxs[idxs[:lim]]
    d2idx = idxs[idxs[lim:]]
    x_train, y_train, x_val, y_val = x[d1idx,:], y[d1idx,:], x[d2idx,:], y[d2idx,:]
else:
    val_probe = good_probes.pop(val_probe)
    
    print('validation probe: %s'%val_probe)

    val_files = list(compress(voltammograms, [x.find(val_probe) > -1 for x in voltammograms]))
    print('number of validation files %d'%len(val_files))
    x_val, y_val = load_matlab(val_files, nrecords_per_session)

    train_files = list(compress(voltammograms, [x.find(val_probe) == -1 for x in voltammograms]))
    print('number of train files %d'%len(train_files))
    x_train, y_train = load_matlab(train_files, nrecords_per_session)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)  

print("--- %s seconds ---" % (time.time() - start_time))


number of voltammograms files 66
after filter 36
after removing bad probes 36
number of train/val files 36
(320760, 999) (320760, 4)
(35640, 999) (35640, 4)
--- 17.064202070236206 seconds ---


In [4]:
# fit GLMNET in parallel (ncores) with cross validation to find lambda
start_time = time.time()
fit = cvglmnet(x = x_train.copy(), y = y_train.copy(), family='mgaussian', parallel=ncores, ptype = 'mse', nfolds = 20)
print("--- %s seconds ---" % (time.time() - start_time))

[status]	Parallel glmnet cv with 28 cores


--- 62750.70887756348 seconds ---


In [5]:
print(fit['lambda_min'])
with open(os.path.join(results_prefix, 'mm_all_pulled_random_folding.pickle'), 'wb') as f:
    pickle.dump(fit, f)

array([0.02503712])

In [14]:
fit.keys()

dict_keys(['lambdau', 'cvm', 'cvsd', 'cvup', 'cvlo', 'nzero', 'name', 'glmnet_fit', 'lambda_min', 'lambda_1se', 'class'])

In [18]:
fit['cvm'].__class__

numpy.ndarray

In [12]:
y_hat = cvglmnetPredict(fit, newx = x_val, s='lambda_min')

In [13]:
for (error, name) in zip(np.mean(np.sqrt((y_hat[:,:,0]-y_val)**2),axis=0),names):
    print('%s: %4.5f'%(name,error))

DA: 617.81527
5HT: 609.45347
pH: 0.10065
NE: 526.68220
